<a href="https://colab.research.google.com/github/RachelNderitu/RachelNderitu/blob/main/Transfer_Learning(Cats_and_Dogs)_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import precision_score, recall_score
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers.experimental.preprocessing import RandomContrast, RandomZoom

In [ ]:
# Download and extract the dataset
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [ ]:
# Define train and validation directories
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

# Define batch size and image size
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

# Load train and validation datasets
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)
validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

class_names = train_dataset.class_names

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [ ]:
# Data preprocessing and augmentation
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# Define data augmentation
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
  RandomContrast(0.1),
  RandomZoom(0.1),
])

# Preprocess input images
preprocess_input = tf.keras.applications.resnet50.preprocess_input

# Load ResNet50 model
base_model = ResNet50(input_shape=IMG_SIZE + (3,),
                      include_top=False,
                      weights='imagenet')

In [ ]:
# Freeze base model layers
base_model.trainable = False

# Create the model
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')])

In [1]:

# Train the model
initial_epochs = 10
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

# Fine-tune the model
base_model.trainable = True
fine_tune_at = 140  # Fine-tuning from this layer onwards
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate / 10),
              metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5, name='accuracy')])

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

NameError: name 'model' is not defined

In [ ]:
# Evaluate the fine-tuned model
loss, accuracy = model.evaluate(validation_dataset)
print("Fine-tuned model validation loss:", loss)
print("Fine-tuned model validation accuracy:", accuracy)

# Calculate precision, recall, and F1-score
predictions = model.predict(validation_dataset)
binary_predictions = (predictions > 0.5).astype(int)
true_labels = np.concatenate([y for x, y in validation_dataset], axis=0)
precision = precision_score(true_labels, binary_predictions)
recall = recall_score(true_labels, binary_predictions)
f1_score = 2 * ((precision * recall) / (precision + recall))

print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1_score)

In [ ]:
# Display images and their predictions
plt.figure(figsize=(10, 10))
for images, labels in validation_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Prediction: {'Dog' if binary_predictions[i] == 1 else 'Cat'}, True Label: {class_names[labels[i]]}")
        plt.axis("off")
plt.show()